In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
import os
import pinecone
from dotenv import load_dotenv

load_dotenv()

True

In [15]:
from langchain.vectorstores import Pinecone
from langchain.embeddings import HuggingFaceEmbeddings
hf_embeddings = HuggingFaceEmbeddings()

index_name = 'langchain-retrieval-agent-sample'

# Load Pinecone API key
api_key = os.getenv('PINECONE_API_KEY') or 'YOUR_API_KEY'
# Set Pinecone environment. Find next to API key in console
env = os.getenv('PINECONE_ENVIRONMENT') or "YOUR_ENV"

pinecone.init(api_key=api_key, environment=env)

text_field = "text"

# switch back to normal index for langchain
index = pinecone.Index(index_name)

vectorstore = Pinecone(
    index, hf_embeddings.embed_query, text_field
)

In [22]:
deployment_name = os.getenv('AZURE_DEPLOYMENT_NAME')

# chat completion llm
llm = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0.0,
    deployment_id = deployment_name
)
# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)
# retrieval qa chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

WARNING! deployment_id is not default parameter.
                    deployment_id was transferred to model_kwargs.
                    Please confirm that deployment_id is what you intended.


In [23]:
query = "How can customers get an exemption from abuse monitoring and human review"

qa.run(query)

"Customers who want to get an exemption from abuse monitoring and human review can apply to modify the Azure OpenAI content management features by completing a form and meeting additional Limited Access eligibility criteria and attest to specific use cases. If Microsoft approves a customer's request to modify abuse monitoring, then Microsoft does not store any prompts and completions associated with the approved Azure subscription for which abuse monitoring is configured off."

In [24]:
from langchain.agents import Tool

tools = [
    Tool(
        name='Knowledge Base',
        func=qa.run,
        description=(
            'use this tool when answering queries about azure openai documentation to get '
            'more information about the topic'
        )
    )
]

In [26]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

In [27]:
agent(query)



> Entering new  chain...
{
    "action": "Knowledge Base",
    "action_input": "How can customers get an exemption from abuse monitoring and human review"
}
Observation: Customers who want to get an exemption from abuse monitoring and human review can apply to modify the Azure OpenAI content management features by completing a form and meeting additional Limited Access eligibility criteria and attest to specific use cases. If Microsoft approves a customer's request to modify abuse monitoring, then Microsoft does not store any prompts and completions associated with the approved Azure subscription for which abuse monitoring is configured off.
Thought:{
    "action": "Final Answer",
    "action_input": "Customers who want to get an exemption from abuse monitoring and human review can apply to modify the Azure OpenAI content management features by completing a form and meeting additional Limited Access eligibility criteria and attest to specific use cases. If Microsoft approves a custom

{'input': 'How can customers get an exemption from abuse monitoring and human review',
 'chat_history': [],
 'output': "Customers who want to get an exemption from abuse monitoring and human review can apply to modify the Azure OpenAI content management features by completing a form and meeting additional Limited Access eligibility criteria and attest to specific use cases. If Microsoft approves a customer's request to modify abuse monitoring, then Microsoft does not store any prompts and completions associated with the approved Azure subscription for which abuse monitoring is configured off."}

In [28]:
agent("what is 2 * 7?")



> Entering new  chain...
{
    "action": "Final Answer",
    "action_input": "14"
}

> Finished chain.


{'input': 'what is 2 * 7?',
 'chat_history': [HumanMessage(content='How can customers get an exemption from abuse monitoring and human review', additional_kwargs={}, example=False),
  AIMessage(content="Customers who want to get an exemption from abuse monitoring and human review can apply to modify the Azure OpenAI content management features by completing a form and meeting additional Limited Access eligibility criteria and attest to specific use cases. If Microsoft approves a customer's request to modify abuse monitoring, then Microsoft does not store any prompts and completions associated with the approved Azure subscription for which abuse monitoring is configured off.", additional_kwargs={}, example=False)],
 'output': '14'}

In [29]:
agent('What data does the Azure OpenAI Service process?')



> Entering new  chain...
{
    "action": "Knowledge Base",
    "action_input": "What data does the Azure OpenAI Service process?"
}
Observation: The Azure OpenAI Service processes prompts to generate content, which can be augmented with relevant data from a configured data store when using the "on your data" feature. Customers can also provide their own training data consisting of prompt-completion pairs for the purposes of fine-tuning an OpenAI model. The service and Microsoft personnel also analyze prompts, completions, and images for harmful content and for patterns suggesting the use of the service in a manner that violates the Code of Conduct or other applicable product terms.
Thought:{
    "action": "Final Answer",
    "action_input": "The Azure OpenAI Service processes prompts to generate content, which can be augmented with relevant data from a configured data store when using the 'on your data' feature. Customers can also provide their own training data consisting of prompt-

{'input': 'What data does the Azure OpenAI Service process?',
 'chat_history': [HumanMessage(content='How can customers get an exemption from abuse monitoring and human review', additional_kwargs={}, example=False),
  AIMessage(content="Customers who want to get an exemption from abuse monitoring and human review can apply to modify the Azure OpenAI content management features by completing a form and meeting additional Limited Access eligibility criteria and attest to specific use cases. If Microsoft approves a customer's request to modify abuse monitoring, then Microsoft does not store any prompts and completions associated with the approved Azure subscription for which abuse monitoring is configured off.", additional_kwargs={}, example=False),
  HumanMessage(content='what is 2 * 7?', additional_kwargs={}, example=False),
  AIMessage(content='14', additional_kwargs={}, example=False)],
 'output': "The Azure OpenAI Service processes prompts to generate content, which can be augmented 